In [1]:
# Import relevant module
import os
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request
from urllib.error import HTTPError
import xarray as xr
from datetime import date, timedelta
import time
import getpass
import lxml
from pydap.client import open_url
from pydap.cas.get_cookies import setup_session

In [ ]:
### This script is not exectuable due to the data set requiring account username and password ###
### Script can be copied with the below code changed to the users personal data ###
### Code base can be found through the Copernicus Marine Service data access information ###

# Create username variable
USERNAME = '###YOUR USER NAME###'
# Create field to enter password and store result
PASSWORD = getpass.getpass('Enter password: ')

In [103]:
# URL to identify desired data via OPeNDAP
DATASET_ID = 'cmems_mod_glo_phy_my_0.083_P1M-m?latitude[0:1:2040],longitude[0:1:4319],depth[0:1:1],so[0:1:328][0:1:1][0:1:2040][0:1:4319],time[0:1:328]'

In [104]:
#### CODE BASE ORGINAL OWNERSHIP ####
__author__ = "Copernicus Marine User Support Team"
__copyright__ = "(C) 2022 E.U. Copernicus Marine Service Information"
__credits__ = ["E.U. Copernicus Marine Service Information"]
__license__ = "MIT License - You must cite this source"
__version__ = "202104"
__maintainer__ = "D. Bazin, E. DiMedio, C. Giordan"
__email__ = "servicedesk dot cmems at mercator hyphen ocean dot eu"

# Function to access data
def copernicusmarine_datastore(dataset, username, password):
    cas_url = 'https://cmems-cas.cls.fr/cas/login' # Login URL
    session = setup_session(cas_url, username, password) # Setup session with credentials
    session.cookies.set("CASTGC", session.cookies.get_dict()['CASTGC']) # Initialise cookies
    database = ['my', 'nrt'] # Database identifiers
    url = f'https://{database[0]}.cmems-du.eu/thredds/dodsC/{dataset}' # URL to try
    
    # Try to retrieve data from first database
    try:
        data_store = xr.backends.PydapDataStore(open_url(url, session=session)) 
    # If error get data from second database
    except:
        url = f'https://{database[1]}.cmems-du.eu/thredds/dodsC/{dataset}'
        data_store = xr.backends.PydapDataStore(open_url(url, session=session)) 
    
    # Return data store variable
    return data_store

In [105]:
# Run function to acquire data
data_store = copernicusmarine_datastore(DATASET_ID, USERNAME, PASSWORD)

In [106]:
# Open dataset
DS = xr.open_dataset(data_store)

In [107]:
# Display data set metadata
DS

<xarray.Dataset>
Dimensions:    (latitude: 2041, longitude: 4320, depth: 2, time: 329)
Coordinates:
  * latitude   (latitude) float32 -80.0 -79.92 -79.83 ... 89.83 89.92 90.0
  * longitude  (longitude) float32 -180.0 -179.9 -179.8 ... 179.8 179.8 179.9
  * depth      (depth) float32 0.494 1.541
  * time       (time) datetime64[ns] 1993-01-16T12:00:00 ... 2020-05-16T12:00:00
Data variables:
    so         (time, depth, latitude, longitude) float32 ...
Attributes: (12/124)
    title:                         Monthly mean fields for product GLOBAL_REA...
    references:                    http://marine.copernicus.eu
    credit:                        E.U. Copernicus Marine Service Information...
    licence:                       http://marine.copernicus.eu/services-portf...
    contact:                       servicedesk.cmems@mercator-ocean.eu
    producer:                      CMEMS - Global Monitoring and Forecasting ...
    ...                            ...
    vo.valid_min:                  -4900
    vo.valid_max:                  4300
    vo.cell_methods:               area: mean
    vo.add_offset:                 0.0
    vo.scale_factor:               0.0006103701889514923
    vo._ChunkSizes:                [1, 7, 341, 720]

In [108]:
# Get data latitudes
ds_lats = DS['latitude'].values[::-1]
# Southern 10 degrees are not covered and need to be added,
# Create list of degrees to add
add_lats = np.array([x/10 for x in range(-805,-905,-10)])
# Combine data set latitudes and latitudes to add
lats = np.concatenate((ds_lats,add_lats), axis=0)

# Get longitudes
lons = DS['longitude'].values

In [109]:
# Need to add NaN values to data where latitudes were missing
# Create list of NaNs
add_lat_nans = np.array([np.nan for x in range(-805,-905,-10)])
# Repeat NaN number of times equal to number of longitude points
full_nans = np.tile(lat_nans, (lons.shape[0],1))
# Transpose to fit with orginal data
full_nans = np.transpose(full_nans)

In [128]:
# Print time metadata
print(DS['time'])

<xarray.DataArray 'time' (time: 329)>
array(['1993-01-16T12:00:00.000000000', '1993-02-15T00:00:00.000000000',
       '1993-03-16T12:00:00.000000000', ..., '2020-03-16T12:00:00.000000000',
       '2020-04-16T00:00:00.000000000', '2020-05-16T12:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1993-01-16T12:00:00 ... 2020-05-16T12:00:00
Attributes:
    axis:           T
    long_name:      Time (hours since 1950-01-01)
    standard_name:  time

In [111]:
### CODE TO REDUCE RESOLUTION TO 1x1 DEGREE RESOLUTION ###

# Set starting index value (allows loop to be done in batches)
start_num = 0
# Loop over shape of salility (equivalent to time indicies)
for i in range(start_num,DS['so'].shape[0],1):
    
    # Print index
    print(i, end=' ')
    # Begin timer to time loop
    start_time = time.time()
    
    # Access data at given index and shallowest depth (index 0 in second position)
    sss_vals = DS['so'][i,0,:,:].values # Get sea surface salinity values
    # Print data confirmation retrival
    print('data retrieved', end=' ->')
    # Flip data to orientate correctly
    sss_vals = np.flip(sss_vals, axis=0)
    # Merge data with array of NaNs 
    sss_vals = np.concatenate((sss_vals,full_nans), axis=0)
    
    # Make dataframe for one month of data indexed by coorinates
    one_month_data = pd.DataFrame(data=sss_vals, index=lats, columns=lons)

    # Initialise a 1x1 degree grid of zeros
    one_deg_grid = np.zeros((len(range(90,-90,-1)),len(range(-180,180,1))))

    # Loop over latitudes - increments of 1-degree
    for lat_ind,latitude in enumerate(range(90,-90,-1)):
        # Loop over longitudes - increments of 1-degree
        for lon_ind,longitude in enumerate(range(-180,180,1)):
            
            # Get 1x1 degree square of values
            # If/elif statements catch map edge values to ensure all data is included
            if (longitude == 179) and (latitude == -89):
                one_deg_square = np.asarray(one_month_data.loc[(one_month_data.index <= latitude) & (one_month_data.index >= latitude-1),(one_month_data.columns >= longitude) & (one_month_data.columns <= longitude+1)])
            
            elif (longitude == 179):
                one_deg_square = np.asarray(one_month_data.loc[(one_month_data.index <= latitude) & (one_month_data.index > latitude-1),(one_month_data.columns >= longitude) & (one_month_data.columns <= longitude+1)])
                
            elif (latitude == -89):
                one_deg_square = np.asarray(one_month_data.loc[(one_month_data.index <= latitude) & (one_month_data.index >= latitude-1),(one_month_data.columns >= longitude) & (one_month_data.columns < longitude+1)])
                
            else:
                one_deg_square = np.asarray(one_month_data.loc[(one_month_data.index <= latitude) & (one_month_data.index > latitude-1),(one_month_data.columns >= longitude) & (one_month_data.columns < longitude+1)])

            # If all values in square are NaN, set the square mean and NaN
            if np.isnan(one_deg_square).all() == True:
                one_deg_mean = np.nan
            # Else set to the mean of the values in the square (excluding NaNs)
            else:
                one_deg_mean = np.nanmean(one_deg_square)

            # Over-write zeros grid with mean at current coordinates
            one_deg_grid[lat_ind, lon_ind] = one_deg_mean
            
    
    # If index is equal to start index then initialise 3D array to store salinity data
    if i == start_num:
        sss_LR_data = one_deg_grid
    # Else append salinity data to 3D array
    else:
        sss_LR_data = np.dstack((sss_LR_data, one_deg_grid))
    
    # End timer of loop
    end_time = time.time()
    # Print shape of data and time of loop as a check
    print(sss_LR_data.shape, round(end_time-start_time,3))
    

264 data retrieved ->(180, 360) 54.587
265 data retrieved ->(180, 360, 2) 51.189
266 data retrieved ->(180, 360, 3) 51.996
267 data retrieved ->(180, 360, 4) 51.827
268 data retrieved ->(180, 360, 5) 52.426
269 data retrieved ->(180, 360, 6) 51.521
270 data retrieved ->(180, 360, 7) 59.854
271 data retrieved ->(180, 360, 8) 52.494
272 data retrieved ->(180, 360, 9) 50.252
273 data retrieved ->(180, 360, 10) 52.249
274 data retrieved ->(180, 360, 11) 52.619
275 data retrieved ->(180, 360, 12) 559.711
276 data retrieved ->(180, 360, 13) 61.473
277 data retrieved ->(180, 360, 14) 50.176
278 data retrieved ->(180, 360, 15) 52.187
279 data retrieved ->(180, 360, 16) 51.348
280 data retrieved ->(180, 360, 17) 56.435
281 data retrieved ->(180, 360, 18) 49.901
282 data retrieved ->(180, 360, 19) 49.906
283 data retrieved ->(180, 360, 20) 50.752
284 data retrieved ->(180, 360, 21) 53.307
285 data retrieved ->(180, 360, 22) 48.699
286 data retrieved ->(180, 360, 23) 49.978
287 data retrieved ->(

In [113]:
# Flatten data for saving to CSV
for d in range(0, sss_LR_data.shape[2]):
    if d == 0:
        sss_LR_flat = sss_LR_data[:,:,d]
    else:
        sss_LR_flat = np.concatenate((sss_LR_flat, sss_LR_data[:,:,d]), axis=0)

# Print flat data array shape as check
print(sss_LR_flat.shape)
# Save data to CSV
np.savetxt('sss_LR_flat.csv',sss_LR_flat, delimiter=',')

### Merge files together ###

In [114]:
# My runs required completing in 4 stages which need combining
# Load low resolution data sets
sss1 = np.loadtxt('sss_LR_flat_0to228.csv', delimiter=',')
sss2 = np.loadtxt('sss_LR_flat_229to231.csv', delimiter=',')
sss3 = np.loadtxt('sss_LR_flat_232to263.csv', delimiter=',')
sss4 = np.loadtxt('sss_LR_flat_264toEND.csv', delimiter=',')

# Concatentate all data sets
sss_all = np.concatenate((sss1,sss2,sss3,sss4),axis=0)
# Print shape of full data set
print(sss_all.shape)

# Save full flat low resolution data
np.savetxt('sss_LR_data.csv', sss_all, delimiter=',')

### Add Climatologies ###

In [2]:
# Load low resolution sea surface salinity data
sss_data = np.loadtxt('sss_LR_data.csv', delimiter=',')

In [3]:
# Convert flat data into 3D array
for r in range(0,sss_data.shape[0],180):
    if r==0:
        sss_stack = sss_data[r:r+180,:]
    else:
        sss_stack = np.dstack((sss_stack,sss_data[r:r+180,:]))

# Print shape of stacked data as a check
print(sss_stack.shape)

(180, 360, 329)


In [4]:
### BACK FILL ###
num_idxs = 15 * 12 # Get number of month indexes for 15 year climatology
# Loop over month positions (0=Jan, 11=Dec)
for d in range(0,12):
    # Get all same month values by indexing every 12 months
    # and get mean of the values over time
    month_climatology = np.nanmean(sss_stack[:,:,d:num_idxs:12],axis=2)
    # If on first month (Jan) initialise 3D array for climatologies
    if d == 0:
        past_climatology = month_climatology
    # Else append climatology to 3D array
    else:
        past_climatology = np.dstack((past_climatology,month_climatology))

# Print shape of data as a check
print(past_climatology.shape)

/Users/joshblannin/opt/anaconda3/envs/python_3_7/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice
  after removing the cwd from sys.path.


In [5]:
### FORWARD FILL ###
num_idxs = (15 * 12) + 1 # Get number of month indexes for 15 year climatology (add one as indexing backwards)
# Loop over month positions (-12=Jun, -1=May) [note Jun=12 becasue this is reverse indexing and last month of data is in May]
for d in range(12,0,-1):
    # Get all same month values by indexing every 12 months
    # and get mean of the values over time
    month_climatology = np.nanmean(sss_stack[:,:,-d:-num_idxs:-12],axis=2)
    # If on first month (Jan) initialise 3D array for climatologies
    if d == 12:
        future_climatology = month_climatology
     # Else append climatology to 3D array
    else:
        future_climatology = np.dstack((future_climatology,month_climatology))

        
# Reorganise data to Jan-Dec by concatenating Jan-May with July-Dec
future_climatology = np.concatenate((future_climatology[:,:,7:], future_climatology[:,:,:7]),axis=2)
# Print shape of data as a check
print(future_climatology.shape)

/Users/joshblannin/opt/anaconda3/envs/python_3_7/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice
  after removing the cwd from sys.path.


In [7]:
# Repeat past data to fill 1990-1992
sss_clim_90to92 = np.tile(past_climatology, (1,1,3))
# Print shape of data as check
print(sss_clim_90to92.shape)

(180, 360, 36)

In [8]:
# Get future climatology from June-Dec
sss_clim_M6toM12 = future_climatology[:,:,5:]
# Print shape of data as check
print(sss_clim_M6toM12.shape)

(180, 360, 7)

In [10]:
# Concatentate 1990-1992 climatologies, orginal data, and Jun-Dec future climatology for complete data set
sss_wClim = np.concatenate((sss_clim_90to92,sss_stack,sss_clim_M6toM12,future_climatology), axis=2)
# Print shape of data as check
print(sss_wClim.shape)

(180, 360, 384)

In [11]:
# Flatten data for writing to CSV
for i in range(0, sss_wClim.shape[2]):
    if i == 0:
        sss_flat = sss_wClim[:,:,i]
    else:
        sss_slice = sss_wClim[:,:,i]
        sss_flat = np.concatenate((sss_flat,sss_slice), axis=0)
        
# Print shape of data as check
print(sss_flat.shape)

# Save data to CSV
np.savetxt('sss_wClimatology.csv', sss_flat, delimiter=',')

(69120, 360)